In [1]:
import numpy as np
import math
import cv2
import tdmclient.notebook
import utils

In [2]:
# await tdmclient.notebook.start(tdm_addr='172.25.112.1', tdm_port=8596)

In [3]:
# await tdmclient.notebook.stop()

In [4]:
# %%run_aseba
# onevent run
#     motor.left.target = 300
#     motor.right.target = 300

# onevent turnleft
#     motor.left.target = -300
#     motor.right.target = 300

# onevent turnright
#     motor.left.target = 300
#     motor.right.target = -300

# onevent stop
#     motor.left.target = 0
#     motor.right.target = 0

Define possible objects and marker colors

In [5]:
ball_blue = [[103, 196, 130], [110, 255, 224]]
red = [[0, 106, 180], [9, 196, 229]]
brown = [[10, 61, 123], [16, 198, 167]]
green = [[31, 78, 114], [65, 199, 193]]
yellow = [[23, 170, 154], [31, 255, 248]]
blue = [[75, 116, 167], [100, 255, 255]]


In [6]:
robot1 = {"front":1, "back":2, "f_rgb": red, "b_rgb": green, "front_loc":(), "back_loc":()}
# robot2 = {"front":3, "back":4, "f_rgb": [[], []], "b_rgb": [[], []], "front_loc":(), "back_loc":()}
ball = {"center":5, "rgb": ball_blue, "loc":()}
goal1 = {"center": 7, "rgb": yellow, "loc":()}
goal2 = {"center": 6, "rgb": blue, "loc":()}
stadium = {"top_left_rgb": brown, "loc": []}

In [7]:
field = {
    "ball": ball,
    "goal1": goal1,
    "goal2": goal2,
}

Define what robots are in the game

In [8]:
robots = {
    "robot1": robot1,
    # "robot2": robot2
}


Variables

In [9]:
COLOR_TOLERANCE: np.array = np.array((2, 25, 25))
DIST_BALL: int = 10

In [10]:
cap = cv2.VideoCapture(0)

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    matrix = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    cv2.imshow('real_frame', frame)
    # Process each robot's front and back points
    for robot, details in robots.items():
        # Detect front points
        f_lower_bound = np.array(details['f_rgb'][0])
        f_upper_bound = np.array(details['f_rgb'][1])
        front_coordinates = utils.detect_colored_dots_in_rgb(image, f_lower_bound, f_upper_bound)
        robots[robot]["front_loc"] = utils.safe_get(front_coordinates, 0, [])

        # Mark front points on the matrix
        for coord in front_coordinates:
            matrix[coord[0], coord[1]] = details['front']
        
        # Detect back points
        b_lower_bound = np.array(details['b_rgb'][0])
        b_upper_bound = np.array(details['b_rgb'][1])
        back_coordinates = utils.detect_colored_dots_in_rgb(image, b_lower_bound, b_upper_bound)
        robots[robot]["back_loc"] = utils.safe_get(back_coordinates, 0, [])

        # Mark back points on the matrix
        for coord in back_coordinates:
            matrix[coord[0], coord[1]] = details['back']
        


    # Find the coordinates of goals and ball
    for feature, details in field.items():
        lower_bound = np.array(details['rgb'][0])
        upper_bound = np.array(details['rgb'][1])
        coordinates = utils.detect_colored_dots_in_rgb(image, lower_bound, upper_bound)
        field[feature]["loc"] = utils.safe_get(coordinates, 0, [])

        for coord in coordinates:
            matrix[coord[0], coord[1]] = details['center']

    #Find the coordinates of stadium top left corner
        # s_tl_lower_bound = np.clip(np.array(stadium['top_left_rgb'])  - COLOR_TOLERANCE, np.array((0, 0, 0)), np.array((180, 255, 255)))
        # s_tl_upper_bound = np.clip(np.array(stadium['top_left_rgb'])  + COLOR_TOLERANCE, np.array((0, 0, 0)), np.array((180, 255, 255)))
        # s_tl_coordinates = utils.detect_colored_dots_in_rgb(image,s_tl_lower_bound, s_tl_upper_bound)

        # s_tr_lower_bound = np.clip(np.array(stadium['top_right_rgb'])  - COLOR_TOLERANCE, np.array((0, 0, 0)), np.array((180, 255, 255)))
        # s_tr_upper_bound = np.clip(np.array(stadium['top_right_rgb'])  + COLOR_TOLERANCE, np.array((0, 0, 0)), np.array((180, 255, 255)))
        # s_tr_coordinates = utils.detect_colored_dots_in_rgb(image,s_tr_lower_bound, s_tr_upper_bound)

        # s_bl_lower_bound = np.clip(np.array(stadium['bottom_left_rgb'])  - COLOR_TOLERANCE, np.array((0, 0, 0)), np.array((180, 255, 255)))
        # s_bl_upper_bound = np.clip(np.array(stadium['bottom_left_rgb'])  + COLOR_TOLERANCE, np.array((0, 0, 0)), np.array((180, 255, 255)))
        # s_bl_coordinates = utils.detect_colored_dots_in_rgb(image,s_bl_lower_bound, s_bl_upper_bound)

        # s_br_lower_bound = np.clip(np.array(stadium['bottom_right_rgb'])  - COLOR_TOLERANCE, np.array((0, 0, 0)), np.array((180, 255, 255)))
        # s_br_upper_bound = np.clip(np.array(stadium['bottom_right_rgb'])  + COLOR_TOLERANCE, np.array((0, 0, 0)), np.array((180, 255, 255)))
        # s_br_coordinates = utils.detect_colored_dots_in_rgb(image,s_br_lower_bound, s_br_upper_bound)

        # s_coords = []

        # s_coords.append(s_tl_coordinates[0])
        # s_coords.append(s_tr_coordinates[0])
        # s_coords.append(s_bl_coordinates[0])
        # s_coords.append(s_br_coordinates[0])

        # stadium["loc"] = s_coords

    #___________________________________________________________TEST_PART______________________________________

    color_map = {
        # 0: (0, 0, 0),       # Black (assuming 0 is the background or no dot)
        1: (255, 0, 0),     # Red
        2: (0, 255, 0),     # Green
        3: (0, 0, 255),     # Blue
        4: (255, 255, 0),   # Cyan
        5: (255, 0, 255),   # Magenta
        6: (0, 255, 255),   # Yellow
        7: (255, 255, 255)  # White
    }

    # Create an empty frame with 3 color channels
    colored_frame = np.zeros((matrix.shape[0], matrix.shape[1], 3), dtype=np.uint8)

    # Draw colored circles in the frame based on matrix values
    for y in range(matrix.shape[0]):
        for x in range(matrix.shape[1]):
            value = matrix[y, x]
            if value in color_map:
                cv2.circle(colored_frame, (x, y), 10, color_map[value], 14)  # Circle with radius 10
    
    shoot_x, shoot_y = utils.ball_shooting_point(field["ball"]["loc"], field["goal2"]["loc"], DIST_BALL)
    print(shoot_x, shoot_y)
    print((shoot_x, shoot_y)[::-1], field["goal2"]["loc"][::-1])
    print(robots["robot1"]["back_loc"][::-1], (shoot_x, shoot_y)[::-1])
    try:
        cv2.line(colored_frame, (shoot_x, shoot_y)[::-1], field["goal2"]["loc"][::-1], (255, 255, 255))
        cv2.line(colored_frame, robots["robot1"]["back_loc"][::-1], (shoot_x, shoot_y)[::-1], (0, 255, 255))
    except:
        pass
    
    colored_frame = cv2.cvtColor(colored_frame, cv2.COLOR_RGB2BGR)
    # Display the frame
    cv2.imshow('Frame', colored_frame)

    dst_points = [[0,0],[0,image.shape[1]],[image.shape[0],0],[image.shape[0],image.shape[1]]]

    # print(stadium["loc"])
    # print(dst_points)
    
    # transformed_image = utils.apply_homography(image,np.float32(stadium["loc"]),np.float32(dst_points))
    # transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
    # cv2.imshow('homography', transformed_image)

    # Wait for 'q' key to quit
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #___________________________________________________________TEST_PART______________________________________
    

cv2.destroyAllWindows()

None None
(None, None) []
[] (None, None)
148 303
(303, 148) (96, 190)
(455, 181) (303, 148)
148 303
(303, 148) (96, 190)
(455, 181) (303, 148)
None None
(None, None) []
(455, 181) (None, None)
None None
(None, None) []
(455, 179) (None, None)
154 328
(328, 154) (96, 190)
(455, 181) (328, 154)
148 312
(312, 148) (96, 190)
(455, 181) (312, 148)
None None
(None, None) []
(455, 181) (None, None)
None None
(None, None) []
(454, 179) (None, None)
None None
(None, None) []
(454, 179) (None, None)
None None
(None, None) []
(455, 179) (None, None)
None None
(None, None) []
(454, 179) (None, None)
146 319
(319, 146) (96, 190)
(455, 181) (319, 146)
148 312
(312, 148) (96, 190)
(455, 181) (312, 148)
154 330
(330, 154) (96, 190)
(454, 179) (330, 154)
153 327
(327, 153) (96, 190)
(434, 184) (327, 153)
None None
(None, None) []
(434, 184) (None, None)
None None
(None, None) []
(455, 181) (None, None)
154 328
(328, 154) (96, 190)
(455, 181) (328, 154)
154 328
(328, 154) (96, 190)
(455, 181) (328, 154

Traceback (most recent call last):
  File "C:\Users\infwo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\infwo\AppData\Local\Temp\ipykernel_53804\575375220.py", line None, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\infwo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 2142, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\infwo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\ultratb.py", li

In [ ]:
cap.release()